In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import copy
import os
from tqdm import tqdm

In [2]:
loadPath = "./Data/Raw/"
loadFile = "SteamAPI_NMS_reviews.xlsx" ## Base file name

In [3]:
dfList = []
loadFileCounter = 0

fileFound = True
while fileFound == True:
    loadFileCounter+=1
    
    ## Proper filename for current loadfile number
    name, fileExt = loadFile.split('.')
    currentloadFile = name + str(loadFileCounter).zfill(3) + '.' + fileExt
    try:
        tmpdf = pd.read_excel(loadPath+currentloadFile,index_col=0)
        fileFound = True
        dfList.append(tmpdf)
    except:
        print(f"No further files detected. Final file {name + str(loadFileCounter-1).zfill(3) + '.' + fileExt}")
        fileFound = False

No further files detected. Final file SteamAPI_NMS_reviews005.xlsx


In [4]:
df = pd.concat(dfList,ignore_index=False)
df

,recommendationid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck
0,198664720,0,1,250,172,243.0,1751392649,english,I liked every thing,2025-07-01 17:47:05,2025-07-01 17:47:05,True,0,0,0.500000,0,True,False,False,False
1,198664615,0,16,16380,1338,16380.0,1751392203,swedish,Good game,2025-07-01 17:45:39,2025-07-01 17:45:39,True,0,0,0.500000,0,True,False,False,False
2,198663932,0,33,14,0,14.0,1672123150,english,No Man in Sky :(,2025-07-01 17:36:47,2025-07-01 17:36:47,True,0,0,0.500000,0,False,False,False,False
3,198663626,0,1,4320,2079,4320.0,1751197679,schinese,\u8fd9\u53ef\u592a\u597d\u73a9\u4e86\uff0c\u67...,2025-07-01 17:33:09,2025-07-01 17:33:09,True,0,0,0.500000,0,False,False,False,False
4,198663578,32,3,1275,1275,1275.0,1751391044,russian,\u043b\u0435\u0442\u0430\u0435\u0448\u044c \u0...,2025-07-01 17:32:26,2025-07-01 17:32:26,True,0,0,0.500000,0,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55180,24845536,56,3,1081,0,136.0,1742339118,schinese,\u671f\u5f85 \u505a\u6c99\u53d1,2016-08-12 17:16:27,2016-08-12 17:16:27,True,0,0,0.500000,0,False,False,False,False
55181,24845492,146,6,27187,0,293.0,1734065603,english,Pretty Darn Good,2016-08-12 17:14:28,2016-08-12 17:14:28,True,0,0,0.450378,0,True,False,False,False
55182,24845487,235,11,933,0,224.0,1618267916,english,Big frame drops and FOV feels too small. Other...,2016-08-12 17:14:03,2016-08-12 17:14:03,True,0,0,0.396006,0,True,False,False,False
55183,24845456,254,1,569,0,364.0,1723120964,tchinese,cannot play!!!!!!!!!!!!!!!!!!!,2016-08-12 17:12:26,2016-08-12 17:12:26,True,4,3,0.493138,0,True,False,False,False


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 355379 entries, 0 to 55184
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   recommendationid             355379 non-null  int64         
 1   num_games_owned              355379 non-null  int64         
 2   num_reviews                  355379 non-null  int64         
 3   playtime_forever             355379 non-null  int64         
 4   playtime_last_two_weeks      355379 non-null  int64         
 5   playtime_at_review           355246 non-null  float64       
 6   last_played                  355379 non-null  int64         
 7   language                     355379 non-null  object        
 8   review                       354299 non-null  object        
 9   timestamp_created            355379 non-null  datetime64[ns]
 10  timestamp_updated            355379 non-null  datetime64[ns]
 11  voted_up                     355

In [6]:
df.describe()

,recommendationid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played,timestamp_created,timestamp_updated,votes_up,votes_funny,weighted_vote_score,comment_count
count,3.553790e+05,355379.000000,355379.000000,3.553790e+05,355379.000000,3.552460e+05,3.553790e+05,355379,355379,355379.000000,3.553790e+05,355379.000000,355379.000000
mean,9.489044e+07,143.274839,17.806668,1.079377e+04,41.935874,5.088608e+03,1.695329e+09,2020-12-29 15:12:40.979635712,2021-05-06 11:59:19.874978560,4.150628,7.251458e+04,0.504683,0.184904
min,2.484539e+07,0.000000,1.000000,0.000000e+00,0.000000,1.000000e+00,0.000000e+00,2016-08-12 17:08:41,2016-08-12 17:08:41,0.000000,0.000000e+00,0.058376,0.000000
25%,4.382008e+07,0.000000,3.000000,1.934000e+03,0.000000,6.730000e+02,1.668295e+09,2018-07-29 17:48:44.500000,2019-02-10 11:27:39,0.000000,0.000000e+00,0.500000,0.000000
50%,8.940440e+07,0.000000,8.000000,5.112000e+03,0.000000,1.939000e+03,1.723771e+09,2021-03-31 01:49:18,2021-08-31 06:13:32,0.000000,0.000000e+00,0.500000,0.000000
75%,1.455082e+08,158.000000,18.000000,1.143100e+04,0.000000,5.079000e+03,1.740872e+09,2023-09-02 08:54:23.500000,2023-11-22 12:58:25,1.000000,0.000000e+00,0.502488,0.000000
max,1.986647e+08,33469.000000,17160.000000,2.309235e+06,20085.000000,1.134557e+06,1.751397e+09,2025-07-01 17:47:05,2025-07-01 17:47:05,16997.000000,4.294967e+09,0.992365,1424.000000
std,5.568367e+07,385.025041,63.577084,2.420035e+04,372.508716,1.293489e+04,6.774500e+07,NaN,NaN,77.498810,1.764764e+07,0.043107,3.788062


In [7]:
df.loc[(df["voted_up"]==False)&(df["timestamp_created"]<="2025-07-02T23")]

,recommendationid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck
46,153383081,0,21,32584,2,32584.0,1751383266,english,All my bases got ruined in the last update whi...,2023-12-11 16:11:33,2025-07-01 15:21:47,False,0,0,0.500000,0,True,False,False,False
59,198649671,48,2,755,755,596.0,1751392242,russian,\u043d\u0435 \u0440\u0435\u043a\u043e\u043c\u0...,2025-07-01 14:40:09,2025-07-01 14:40:09,False,0,0,0.500000,0,True,False,False,False
61,198649396,0,7,15676,3178,15676.0,1751378938,dutch,first 2 hours are fun then it becomes rapidely...,2025-07-01 14:36:36,2025-07-01 14:36:36,False,0,0,0.500000,0,True,False,False,False
144,198609283,0,33,580,0,580.0,1736495198,tchinese,\u9019\u904a\u6232\u4e0d\u9069\u5408\u5bc2\u5b...,2025-07-01 03:16:39,2025-07-01 03:16:39,False,0,0,0.476190,0,True,False,False,False
148,198606600,1419,383,558,558,213.0,1751386506,schinese,\u672c\u8d28\u4e0a\u5c31\u662f\u4e2a\u79d1\u5e...,2025-07-01 02:27:25,2025-07-01 02:27:25,False,0,0,0.418251,0,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55155,24845735,0,3,2372,0,136.0,1732483530,english,This looks like it was made for 5 year olds. \...,2016-08-12 17:25:25,2016-08-12 17:25:25,False,1,0,0.503817,2,False,False,False,False
55158,24845726,0,7,36,0,16.0,1532223509,english,runs like shit and its just a crappy console p...,2016-08-12 17:25:07,2016-08-12 17:25:07,False,0,0,0.500000,0,True,False,False,False
55159,24845723,0,6,832,0,306.0,1695575231,schinese,\u6709\u6ca1\u6709\u641e\u9519\uff0c\u6e38\u62...,2016-08-12 17:25:00,2016-08-12 17:25:00,False,0,0,0.500000,0,True,False,False,False
55164,24845698,0,1,15,0,15.0,1471061154,english,can't launch the gaaaaame!!!!,2016-08-12 17:24:04,2016-08-12 17:24:04,False,1,0,0.500216,0,True,False,False,False


In [8]:
df.loc[(df["review"].str.contains(r"\\u[0-9a-zA-Z]+",flags=re.IGNORECASE,regex=True,na=False))]

,recommendationid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck
3,198663626,0,1,4320,2079,4320.0,1751197679,schinese,\u8fd9\u53ef\u592a\u597d\u73a9\u4e86\uff0c\u67...,2025-07-01 17:33:09,2025-07-01 17:33:09,True,0,0,0.500000,0,False,False,False,False
4,198663578,32,3,1275,1275,1275.0,1751391044,russian,\u043b\u0435\u0442\u0430\u0435\u0448\u044c \u0...,2025-07-01 17:32:26,2025-07-01 17:32:26,True,0,0,0.500000,0,True,False,False,False
6,198662642,372,14,14120,0,14120.0,1631533284,schinese,[strike] \u4f60\u662f\u600e\u4e48\u505a\u5230\...,2025-07-01 17:20:03,2025-07-01 17:20:03,True,0,0,0.500000,0,True,False,False,False
8,76573425,0,14,17001,0,17001.0,1741762335,russian,\u0445\u043e\u0440\u043e\u0448\u043e \u043f\u0...,2020-09-26 03:27:25,2025-07-01 17:12:37,True,0,0,0.476190,0,True,False,False,False
9,198662008,0,1,776,776,746.0,1751391718,russian,\u041d\u0435\u043d\u0430\u043f\u0440\u044f\u04...,2025-07-01 17:12:15,2025-07-01 17:12:15,True,0,0,0.500000,0,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55170,24845621,0,987,3231,0,712.0,1721323226,english,"This games been getting a lot of hatred, and i...",2016-08-12 17:20:48,2016-08-12 17:20:48,True,213,15,0.393053,21,True,False,False,False
55177,24845546,0,67,1854,0,545.0,1690338563,english,Bought No Man\u2019s Sky \nInstalled No Man\...,2016-08-12 17:17:01,2016-08-12 17:17:01,True,8,22,0.367425,4,True,False,False,False
55178,24845541,637,18,1979,0,175.0,1672137869,schinese,\u6211\u8981\u98de\u7684\u66f4\u8fdc~~\n\u98de...,2016-08-12 17:16:42,2016-08-12 17:16:42,True,3,2,0.507152,0,False,False,False,False
55180,24845536,56,3,1081,0,136.0,1742339118,schinese,\u671f\u5f85 \u505a\u6c99\u53d1,2016-08-12 17:16:27,2016-08-12 17:16:27,True,0,0,0.500000,0,False,False,False,False


In [9]:
def df_series_strip(df, target, replacement=r"",compare=False):
    modDF = df.copy()
    modDF = modDF.replace(to_replace=target,value=replacement,regex=True)

    if compare:
        for key, val in target.items():
            print("Before vs after")
            print(df[key].loc[(df[key].str.contains(val,flags=re.IGNORECASE,regex=True,na=False))].head(n=5))
            print(modDF[key].loc[(df[key].str.contains(val,flags=re.IGNORECASE,regex=True,na=False))].head(n=5))
            print("\n")
    return modDF

In [10]:
modDF = df_series_strip(df,target={"review":r"\\[uU]+[0-9a-zA-Z]+"},compare=True)

Before vs after
3    \u8fd9\u53ef\u592a\u597d\u73a9\u4e86\uff0c\u67...
4    \u043b\u0435\u0442\u0430\u0435\u0448\u044c \u0...
6    [strike] \u4f60\u662f\u600e\u4e48\u505a\u5230\...
8    \u0445\u043e\u0440\u043e\u0448\u043e \u043f\u0...
9    \u041d\u0435\u043d\u0430\u043f\u0440\u044f\u04...
Name: review, dtype: object
3                                     
4                                -.   
6                  [strike]  [/strike]
8                                    !
9    , ,   . , , ...   ...      )))  !
Name: review, dtype: object




In [11]:
modDF = df_series_strip(modDF,target={"review":r"[\[]{1}[\/]?strike[\]]{1}"},compare=True)

Before vs after
6                                     [strike]  [/strike]
2366    [strike]After 73 hour xbox controller stops wo...
2504    Leave the convergence. Help the Autophage. let...
2608    [strike] Quase 10 anos de desenvolvimento e ai...
2685    [strike] So the out of control hype levels for...
Name: review, dtype: object
6                                                        
2366    After 73 hour xbox controller stops working pr...
2504    Leave the convergence. Help the Autophage. let...
2608     Quase 10 anos de desenvolvimento e ainda tem ...
2685     So the out of control hype levels for this ga...
Name: review, dtype: object




In [12]:
modDF = df_series_strip(modDF,target={"review":r"\\n"},compare=True)

Before vs after
15    Gets better and better as you play.\r\nGOOD GAME!
17      ,  :       .    ,   .     ,  . , ,  ,       ...
33          :  ,   ,  ,  .\n   ,    .     .\n  ,    ...
40    Beautiful game with lots of content. If you en...
41            Han mejorado esta mierda much\xedsimo\r\n
Name: review, dtype: object
15      Gets better and better as you play.\rGOOD GAME!
17      ,  :       .    ,   .     ,  . , ,  ,       ...
33          :  ,   ,  ,  .   ,    .     .  ,        ...
40    Beautiful game with lots of content. If you en...
41              Han mejorado esta mierda much\xedsimo\r
Name: review, dtype: object




In [13]:
modDF = df_series_strip(modDF,target={"review":r"\\r"},compare=True)

Before vs after
15      Gets better and better as you play.\rGOOD GAME!
40    Beautiful game with lots of content. If you en...
41              Han mejorado esta mierda much\xedsimo\r
53    Amazing Game!\rHesitated for years to get it.\...
55                          ,   ,         .\r\r 10   10
Name: review, dtype: object
15        Gets better and better as you play.GOOD GAME!
40    Beautiful game with lots of content. If you en...
41                Han mejorado esta mierda much\xedsimo
53    Amazing Game!Hesitated for years to get it.Now...
55                              ,   ,         . 10   10
Name: review, dtype: object




In [14]:
modDF = df_series_strip(modDF,target={"review":r"[\s]{2,}"},compare=True)

Before vs after
4                                 -.   
6                                      
8                                     !
9     , ,   . , , ...   ...      )))  !
10                                 !  !
Name: review, dtype: object
4                      -.
6                        
8                       !
9     , ,. , , ......)))!
10                     !!
Name: review, dtype: object




In [15]:
modDF = df_series_strip(modDF,target={"review":r"[^\w]{3,}"},replacement=r" ",compare=True)

Before vs after
2                                      No Man in Sky :(
4                                                    -.
5     This. Is. Too good for words to say. BUT, ther...
9                                   , ,. , , ......)))!
17    ,:.,.,. , ,,.,,:Satisfactory,.- ,,. ,.... .,.,...
Name: review, dtype: object
2                                        No Man in Sky 
4                                                      
5     This. Is. Too good for words to say. BUT, ther...
9                                                      
17                      Satisfactory No Man Sky 9/1010.
Name: review, dtype: object




In [16]:
modDF.loc[modDF["review"]=="","review"] = pd.NA
modDF.loc[modDF["review"]==" ","review"] = pd.NA
whereBefore = modDF["review"].isnull()==True
copyBefore = modDF.copy()

In [17]:
modDF["review"] = modDF["review"].str.strip(".-,;:!?()#*&^%'@~[]{}+=#_/\£$`¬'")
modDF["review"] = modDF["review"].str.strip()
modDF.loc[modDF["review"]=="","review"] = pd.NA
modDF.loc[modDF["review"]==" ","review"] = pd.NA
whereAfter = modDF["review"].isnull()==True
modDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 355379 entries, 0 to 55184
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   recommendationid             355379 non-null  int64         
 1   num_games_owned              355379 non-null  int64         
 2   num_reviews                  355379 non-null  int64         
 3   playtime_forever             355379 non-null  int64         
 4   playtime_last_two_weeks      355379 non-null  int64         
 5   playtime_at_review           355246 non-null  float64       
 6   last_played                  355379 non-null  int64         
 7   language                     355379 non-null  object        
 8   review                       286985 non-null  object        
 9   timestamp_created            355379 non-null  datetime64[ns]
 10  timestamp_updated            355379 non-null  datetime64[ns]
 11  voted_up                     355

In [18]:
copyBefore["review"].loc[(whereBefore==False)&(whereAfter==True)]

8         !
10       !!
21       =)
27        ,
67        .
         ..
55002    .(
55118    ()
55131    ~~
55163    ,.
55184     .
Name: review, Length: 7948, dtype: object

In [19]:
modDF = modDF.drop_duplicates()
modDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 355369 entries, 0 to 55184
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   recommendationid             355369 non-null  int64         
 1   num_games_owned              355369 non-null  int64         
 2   num_reviews                  355369 non-null  int64         
 3   playtime_forever             355369 non-null  int64         
 4   playtime_last_two_weeks      355369 non-null  int64         
 5   playtime_at_review           355236 non-null  float64       
 6   last_played                  355369 non-null  int64         
 7   language                     355369 non-null  object        
 8   review                       286978 non-null  object        
 9   timestamp_created            355369 non-null  datetime64[ns]
 10  timestamp_updated            355369 non-null  datetime64[ns]
 11  voted_up                     355

In [20]:
def write_to_excel(df,savePath,saveFile):
    tmp = ""
    for savePathChunk in savePath.split("/")[:-1]:
        tmp += savePathChunk + "/"
        try:
            os.mkdir(tmp)
        except:
            pass

    path = savePath+saveFile
    with pd.ExcelWriter(path=path,mode="w") as writer:
        df.to_excel(writer,engine='xlsxwriter')
    print(f"{path}")
    return

In [21]:
savePath = "./Data/"
saveFile = "SteamAPI_NMS_reviews.xlsx"

try:
    del df
except:
    pass

try:
    del tmp
except:
    pass
    
write_to_excel(modDF,savePath,saveFile)

./Data/SteamAPI_NMS_reviews.xlsx
